In [187]:
import pandas as pd
import sqlite3
from sqlite3 import Error
import os
import csv
from datetime import datetime

def create_connection(db_file, delete_db=False):
    """Creates a connection to SQLite DB with optional delete flag"""
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
        return conn
    except sqlite3.Error as e:
        print(e)
        return None

def create_table(conn, create_table_sql, drop_table_name=None):
    """Drops and creates a table in SQLite"""
    try:
        if drop_table_name:
            conn.execute(f"DROP TABLE IF EXISTS {drop_table_name}")
        conn.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)

In [273]:
#CREATE TABLE Country (
#    CountryID INTEGER PRIMARY KEY AUTOINCREMENT,
#    Country TEXT NOT NULL
#    )
#
#CREATE TABLE Region (
#        RegionID INTEGER PRIMARY KEY AUTOINCREMENT,
#        Region TEXT NOT NULL,
#        CountryID INTEGER NOT NULL,
#        FOREIGN KEY (CountryID) REFERENCES Country(CountryID)
#    )
#
#CREATE TABLE State (
#    StateID INTEGER PRIMARY KEY AUTOINCREMENT,
#    State TEXT NOT NULL,
#    RegionID INTEGER NOT NULL,
#    FOREIGN KEY (RegionID) REFERENCES Region(RegionID)
#)
#
#CREATE TABLE Customer (
#    CustomerID Integer NOT NULL PRIMARY KEY AUTOINCREMENT,
#    FirstName TEXT NOT NULL,
#    LastName TEXT NOT NULL,
#    City TEXT NOT NULL,
#    StateID INTEGER NOT NULL,
#    FOREIGN KEY (StateID) REFERENCES State(StateID)
#)
#
#CREATE TABLE Category (
#    CategoryID INTEGER PRIMARY KEY AUTOINCREMENT,
#    Category TEXT NOT NULL
#)
#
#CREATE TABLE SubCategory (
#    SubCategoryID INTEGER PRIMARY KEY AUTOINCREMENT,
#    SubCategory TEXT NOT NULL,
#    CategoryID INTEGER NOT NULL,
#    FOREIGN KEY (CategoryID) REFERENCES Category(CategoryID)
#)
#
#CREATE TABLE Product (
#    ProductID TEXT PRIMARY KEY,
#    ProductName TEXT NOT NULL,
#    SubCategoryID INTEGER NOT NULL,
#    Price REAL NOT NULL,
#    FOREIGN KEY (SubCategoryID) REFERENCES SubCategory(SubCategoryID)
#)
#
#CREATE TABLE Orders (
#    OrderID TEXT NOT NULL PRIMARY KEY,
#    CustomerID TEXT NOT NULL,
#    ProductID TEXT NOT NULL,
#    OrderDate TEXT NOT NULL,
#    Quantity INTEGER NOT NULL,
#    Sales REAL NOT NULL,
#    FOREIGN KEY (CustomerID) REFERENCES Customer(CustomerID),
#    FOREIGN KEY (ProductID) REFERENCES Product(ProductID)
#);


Creating Country Table

In [33]:
def step1_create_country_table(data_filename, db_file):
    countries = set()
    with open(data_filename, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            country = row['Country'].strip()
            if country:
                countries.add((country,))
    countries = sorted(list(countries), key=lambda x: x[0])

    create_country_table_sql = """
    CREATE TABLE Country (
        CountryID INTEGER PRIMARY KEY AUTOINCREMENT,
        Country TEXT NOT NULL
    );
    """

    def insert_country(conn, values):
        sql = "INSERT INTO Country (Country) VALUES (?)"
        conn.executemany(sql, values)

    conn = create_connection(db_file)
    with conn:
        create_table(conn, create_country_table_sql, "Country")
        insert_country(conn, countries)
    conn.close()

In [35]:
step1_create_country_table("Sales_data.csv", "normalized_sales.db")

In [37]:
conn = sqlite3.connect("normalized_sales.db")
df = pd.read_sql_query("select * from Country", conn)
display(df)
conn.close()

,CountryID,Country
0,1,United States


In [39]:
def step2_create_country_to_countryid_dictionary(db_file):
    conn = create_connection(db_file)
    df = pd.read_sql_query("SELECT * FROM Country", conn)
    country_dict = {row["Country"]: row["CountryID"] for _, row in df.iterrows()}
    conn.close()
    return country_dict

In [41]:
country_map = step2_create_country_to_countryid_dictionary("normalized_sales.db")
print(country_map)

{'United States': 1}


Creating Region Table

In [43]:
def step3_create_region_table(data_filename, db_file):
    country_to_id = step2_create_country_to_countryid_dictionary(db_file)
    regions = set()

    with open(data_filename, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            region = row['Region'].strip()
            country = row['Country'].strip()
            country_id = country_to_id.get(country)
            if region and country_id:
                regions.add((region, country_id))
    regions = sorted(list(regions), key=lambda x: x[0])

    create_region_table_sql = """
    CREATE TABLE Region (
        RegionID INTEGER PRIMARY KEY AUTOINCREMENT,
        Region TEXT NOT NULL,
        CountryID INTEGER NOT NULL,
        FOREIGN KEY (CountryID) REFERENCES Country(CountryID)
    );
    """

    def insert_region(conn, values):
        sql = "INSERT INTO Region (Region, CountryID) VALUES (?, ?)"
        conn.executemany(sql, values)

    conn = create_connection(db_file)
    with conn:
        create_table(conn, create_region_table_sql, "Region")
        insert_region(conn, regions)
    conn.close()


In [45]:
step3_create_region_table("Sales_data.csv", "normalized_sales.db")

In [47]:
conn = sqlite3.connect("normalized_sales.db")
df = pd.read_sql_query("select * from Region", conn)
display(df)
conn.close()

,RegionID,Region,CountryID
0,1,Central,1
1,2,East,1
2,3,South,1
3,4,West,1


In [49]:
# Step 4: Region → RegionID dictionary
def step4_create_region_to_regionid_dictionary(db_file):
    conn = create_connection(db_file)
    df = pd.read_sql_query("SELECT * FROM Region", conn)
    region_dict = {row["Region"]: row["RegionID"] for _, row in df.iterrows()}
    conn.close()
    return region_dict

In [51]:
region_map = step4_create_region_to_regionid_dictionary("normalized_sales.db")
print(region_map)

{'Central': 1, 'East': 2, 'South': 3, 'West': 4}


Creating State Table

In [53]:
def step5_create_state_table(data_filename, db_file):
    region_to_id = step4_create_region_to_regionid_dictionary(db_file)
    states = set()

    with open(data_filename, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            state = row['State'].strip()
            region = row['Region'].strip()
            region_id = region_to_id.get(region)
            if state and region_id:
                states.add((state, region_id))
    states = sorted(list(states), key=lambda x: x[0])

    create_state_table_sql = """
    CREATE TABLE State (
        StateID INTEGER PRIMARY KEY AUTOINCREMENT,
        State TEXT NOT NULL,
        RegionID INTEGER NOT NULL,
        FOREIGN KEY (RegionID) REFERENCES Region(RegionID)
    );
    """

    def insert_state(conn, values):
        sql = "INSERT INTO State (State, RegionID) VALUES (?, ?)"
        conn.executemany(sql, values)

    conn = create_connection(db_file)
    with conn:
        create_table(conn, create_state_table_sql, "State")
        insert_state(conn, states)
    conn.close()

In [55]:
step5_create_state_table("Sales_data.csv", "normalized_sales.db")

In [57]:
conn = sqlite3.connect("normalized_sales.db")
df = pd.read_sql_query("select * from State", conn)
display(df)
conn.close()

,StateID,State,RegionID
0,1,Alabama,3
1,2,Arizona,4
2,3,Arkansas,3
3,4,California,4
4,5,Colorado,4
5,6,Connecticut,2
6,7,Delaware,2
7,8,District of Columbia,2
8,9,Florida,3
9,10,Georgia,3


In [59]:
def step6_create_state_to_stateid_dictionary(db_file):
    conn = create_connection(db_file)
    df = pd.read_sql_query("SELECT * FROM State", conn)
    state_dict = {row["State"]: row["StateID"] for _, row in df.iterrows()}
    conn.close()
    return state_dict

In [61]:
state_map = step6_create_state_to_stateid_dictionary("normalized_sales.db")
print(state_map)

{'Alabama': 1, 'Arizona': 2, 'Arkansas': 3, 'California': 4, 'Colorado': 5, 'Connecticut': 6, 'Delaware': 7, 'District of Columbia': 8, 'Florida': 9, 'Georgia': 10, 'Idaho': 11, 'Illinois': 12, 'Indiana': 13, 'Iowa': 14, 'Kansas': 15, 'Kentucky': 16, 'Louisiana': 17, 'Maine': 18, 'Maryland': 19, 'Massachusetts': 20, 'Michigan': 21, 'Minnesota': 22, 'Mississippi': 23, 'Missouri': 24, 'Montana': 25, 'Nebraska': 26, 'Nevada': 27, 'New Hampshire': 28, 'New Jersey': 29, 'New Mexico': 30, 'New York': 31, 'North Carolina': 32, 'North Dakota': 33, 'Ohio': 34, 'Oklahoma': 35, 'Oregon': 36, 'Pennsylvania': 37, 'Rhode Island': 38, 'South Carolina': 39, 'South Dakota': 40, 'Tennessee': 41, 'Texas': 42, 'Utah': 43, 'Vermont': 44, 'Virginia': 45, 'Washington': 46, 'West Virginia': 47, 'Wisconsin': 48, 'Wyoming': 49}


Creating Customer Table

In [93]:
def step7_create_customer_table(data_filename, db_file):
    state_to_id = step6_create_state_to_stateid_dictionary(db_file)
    customers = set()

    with open(data_filename, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            full_name = row['Customer Name'].strip()
            city = row['City'].strip()
            state = row['State'].strip()
            state_id = state_to_id.get(state)

            if not state_id:
                continue

            name_parts = full_name.split(' ', 1)
            first_name = name_parts[0]
            last_name = name_parts[1] if len(name_parts) > 1 else ''
            customer_tuple = (first_name, last_name, city, state_id)
            customers.add(customer_tuple)

    customers = sorted(list(customers), key=lambda x: x[0])

    create_customer_table_sql = """
    CREATE TABLE Customer (
        CustomerID Integer NOT NULL PRIMARY KEY AUTOINCREMENT,
        FirstName TEXT NOT NULL,
        LastName TEXT NOT NULL,
        City TEXT NOT NULL,
        StateID INTEGER NOT NULL,
        FOREIGN KEY (StateID) REFERENCES State(StateID)
    );
    """

    def insert_customer(conn, values):
        sql = """
        INSERT INTO Customer (FirstName, LastName, City, StateID)
        VALUES (?, ?, ?, ?)
        """
        conn.executemany(sql, values)

    conn = create_connection(db_file)
    with conn:
        create_table(conn, create_customer_table_sql, "Customer")
        insert_customer(conn, customers)
    conn.close()

    print("Customer table created and populated with StateID.")


In [95]:
step7_create_customer_table("Sales_data.csv", "normalized_sales.db")

Customer table created and populated with StateID.


In [97]:
conn = sqlite3.connect("normalized_sales.db")
df = pd.read_sql_query("select * from Customer", conn)
display(df)
conn.close()

,CustomerID,FirstName,LastName,City,StateID
0,1,Aaron,Hawkins,New York City,31
1,2,Aaron,Hawkins,Gulfport,23
2,3,Aaron,Bergman,Arlington,42
3,4,Aaron,Smayling,Jacksonville,32
4,5,Aaron,Hawkins,Troy,31
...,...,...,...,...,...
4609,4610,Zuschuss,Carroll,Salem,36
4610,4611,Zuschuss,Carroll,Miami,9
4611,4612,Zuschuss,Carroll,Aurora,12
4612,4613,Zuschuss,Donatelli,Los Angeles,4


In [99]:
def step8_create_customer_to_customerid_dictionary(db_file):

    conn = create_connection(db_file)
    df = pd.read_sql_query("SELECT CustomerID, FirstName, LastName FROM Customer", conn)

    df['FullName'] = df['FirstName'] + ' ' + df['LastName']
    customer_dict = {row['FullName'].strip(): row['CustomerID'] for _, row in df.iterrows()}

    conn.close()
    return customer_dict

In [101]:
customer_map = step8_create_customer_to_customerid_dictionary("normalized_sales.db")
print(customer_map)

{'Aaron Hawkins': 9, 'Aaron Bergman': 15, 'Aaron Smayling': 16, 'Adam Hart': 41, 'Adam Shillingsburg': 35, 'Adam Bellavance': 40, 'Adrian Shami': 54, 'Adrian Barton': 57, 'Adrian Hane': 59, 'Aimee Bixby': 64, 'Alan Shonely': 91, 'Alan Dominguez': 98, 'Alan Schoenberger': 97, 'Alan Barnes': 99, 'Alan Haines': 82, 'Alan Hwang': 101, 'Alejandro Ballentine': 119, 'Alejandro Savely': 120, 'Alejandro Grove': 116, 'Aleksandra Gannaway': 124, 'Alex Russell': 132, 'Alex Grayson': 136, 'Alex Avila': 137, 'Alice McCarthy': 141, 'Allen Goldenen': 159, 'Allen Rosenblatt': 160, 'Allen Armold': 158, 'Alyssa Crouse': 164, 'Alyssa Tate': 166, 'Amy Cox': 178, 'Amy Hunt': 176, 'Andrew Gjertsen': 194, 'Andrew Allen': 192, 'Andrew Roberts': 193, 'Andy Reiter': 208, 'Andy Yotov': 206, 'Andy Gerbode': 205, 'Anemone Ratner': 209, 'Angele Hood': 212, 'Ann Chong': 227, 'Ann Steele': 224, 'Ann Blume': 228, 'Anna Chung': 250, 'Anna Gayman': 255, 'Anna Häberlin': 257, 'Anna Andreadi': 252, 'Anne McFarland': 270, '

Creating Category Table

In [103]:
def step10_create_category_table(data_filename, db_file):
    categories = set()
    with open(data_filename, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            category = row['Category'].strip()
            if category:
                categories.add((category,))

    categories = sorted(list(categories), key=lambda x: x[0])  # sort alphabetically

    create_category_table_sql = """
    CREATE TABLE Category (
        CategoryID INTEGER PRIMARY KEY AUTOINCREMENT,
        Category TEXT NOT NULL
    );
    """
    def insert_categories(conn, values):
        sql = "INSERT INTO Category (Category) VALUES (?)"
        conn.executemany(sql, values)

    conn = create_connection(db_file)
    with conn:
        create_table(conn, create_category_table_sql, "Category")
        insert_categories(conn, categories)
    conn.close()

    print("Category table created and populated.")

In [105]:
step10_create_category_table("Sales_data.csv", "normalized_sales.db")

Category table created and populated.


In [107]:
conn = sqlite3.connect("normalized_sales.db")
df = pd.read_sql_query("select * from Category", conn)
display(df)
conn.close()

,CategoryID,Category
0,1,Furniture
1,2,Office Supplies
2,3,Technology


In [109]:
def step11_create_category_to_categoryid_dictionary(db_file):

    conn = create_connection(db_file)
    df = pd.read_sql_query("SELECT * FROM Category", conn)
    conn.close()

    category_dict = {row["Category"]: row["CategoryID"] for _, row in df.iterrows()}
    return category_dict

In [111]:
category_map = step11_create_category_to_categoryid_dictionary("normalized_sales.db")
print(category_map)

{'Furniture': 1, 'Office Supplies': 2, 'Technology': 3}


Creating Sub-Category Table

In [233]:
def step12_create_subcategory_table_with_categoryid(data_filename, db_file):
    category_to_id = step11_create_category_to_categoryid_dictionary(db_file)
    subcategories = set()

    with open(data_filename, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            subcategory = row['Sub-Category'].strip()
            category = row['Category'].strip()
            category_id = category_to_id.get(category)
            if subcategory and category_id:
                subcategories.add((subcategory, category_id))

    subcategories = sorted(list(subcategories), key=lambda x: x[0])

    # Define table
    create_subcategory_table_sql = """
    CREATE TABLE SubCategory (
        SubCategoryID INTEGER PRIMARY KEY AUTOINCREMENT,
        SubCategory TEXT NOT NULL,
        CategoryID INTEGER NOT NULL,
        FOREIGN KEY (CategoryID) REFERENCES Category(CategoryID)
    );
    """
    
    def insert_subcategories(conn, values):
        sql = "INSERT INTO SubCategory (SubCategory, CategoryID) VALUES (?, ?)"
        conn.executemany(sql, values)


    conn = create_connection(db_file)
    with conn:
        create_table(conn, create_subcategory_table_sql, "SubCategory")
        insert_subcategories(conn, subcategories)
    conn.close()

    print("SubCategory table created and linked to CategoryID.")


In [235]:
step12_create_subcategory_table_with_categoryid("Sales_data.csv", "normalized_sales.db")

FOREIGN KEY constraint failed
SubCategory table created and linked to CategoryID.


In [237]:
conn = sqlite3.connect("normalized_sales.db")
df = pd.read_sql_query("select * from SubCategory", conn)
display(df)
conn.close()

,SubCategoryID,SubCategory,CategoryID
0,1,Accessories,3
1,2,Appliances,2
2,3,Art,2
3,4,Binders,2
4,5,Bookcases,1
5,6,Chairs,1
6,7,Copiers,3
7,8,Envelopes,2
8,9,Fasteners,2
9,10,Furnishings,1


In [127]:
def step13_create_subcategory_to_id_dict(db_file):
    conn = sqlite3.connect(db_file)
    df = pd.read_sql_query("SELECT * FROM SubCategory", conn)
    conn.close()
    subcategory_dict = {row["SubCategory"]: row["SubCategoryID"] for _, row in df.iterrows()}
    return subcategory_dict

In [129]:
subcategory_map = step13_create_subcategory_to_id_dict("normalized_sales.db")
print(subcategory_map)

{'Accessories': 1, 'Appliances': 2, 'Art': 3, 'Binders': 4, 'Bookcases': 5, 'Chairs': 6, 'Copiers': 7, 'Envelopes': 8, 'Fasteners': 9, 'Furnishings': 10, 'Labels': 11, 'Machines': 12, 'Paper': 13, 'Phones': 14, 'Storage': 15, 'Supplies': 16, 'Tables': 17}


Creating Product Table

In [155]:
def step14_create_product_table(data_filename, db_file):
   
    def insert_products(conn, values):
        sql = """
        INSERT INTO Product (ProductID, ProductName, SubCategoryID, Price)
        VALUES (?, ?, ?, ?)
        """
        conn.executemany(sql, values)

    subcategory_to_id = step_create_subcategory_to_id_dict(db_file)

    # Use a dictionary to filter unique ProductIDs
    unique_products = {}

    with open(data_filename, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            product_id = row['Product ID'].strip()
            if product_id in unique_products:
                continue  # Skip duplicates

            product_name = row['Product Name'].strip()
            subcategory = row['Sub-Category'].strip()
            price = float(row['Price'])

            subcategory_id = subcategory_to_id.get(subcategory)
            if product_id and product_name and subcategory_id:
                unique_products[product_id] = (product_id, product_name, subcategory_id, price)

    product_records = sorted(list(unique_products.values()), key=lambda x: x[0])

    create_product_table_sql = """
    CREATE TABLE Product (
        ProductID TEXT PRIMARY KEY,
        ProductName TEXT NOT NULL,
        SubCategoryID INTEGER NOT NULL,
        Price REAL NOT NULL,
        FOREIGN KEY (SubCategoryID) REFERENCES SubCategory(SubCategoryID)
    );
    """

    conn = create_connection(db_file)
    with conn:
        create_table(conn, create_product_table_sql, "Product")
        insert_products(conn, product_records)
    conn.close()

    print("Product table created with unique ProductIDs and linked to SubCategoryID.")



In [159]:
step14_create_product_table("Sales_data.csv", "normalized_sales.db")

Product table created with unique ProductIDs and linked to SubCategoryID.


In [161]:
conn = sqlite3.connect("normalized_sales.db")
df = pd.read_sql_query("select * from Product", conn)
display(df)
conn.close()

,ProductID,ProductName,SubCategoryID,Price
0,FUR-BO-10000112,"Bush Birmingham Collection Bookcase, Dark Cherry",5,61.67
1,FUR-BO-10000330,"Sauder Camden County Barrister Bookcase, Plank...",5,546.36
2,FUR-BO-10000362,Sauder Inglewood Library Bookcases,5,467.33
3,FUR-BO-10000468,O'Sullivan 2-Shelf Heavy-Duty Bookcases,5,511.26
4,FUR-BO-10000711,"Hon Metal Bookcases, Gray",5,868.71
...,...,...,...,...
1856,TEC-PH-10004912,Cisco SPA112 2 Port Phone Adapter,14,343.56
1857,TEC-PH-10004922,RCA Visys Integrated PBX 8-Line Router,14,683.36
1858,TEC-PH-10004924,"SKILCRAFT Telephone Shoulder Rest, 2"" x 6.5"" x...",14,438.95
1859,TEC-PH-10004959,Classic Ivory Antique Telephone ZL1810,14,908.92


In [163]:
def step15_create_product_to_productid_dictionary(db_file):

    conn = sqlite3.connect(db_file)
    df = pd.read_sql_query("SELECT ProductID, ProductName FROM Product", conn)
    conn.close()

    product_dict = {row["ProductName"]: row["ProductID"] for _, row in df.iterrows()}
    return product_dict

In [165]:
product_map = step15_create_product_to_productid_dictionary("normalized_sales.db")
print(product_map)

{'Bush Birmingham Collection Bookcase, Dark Cherry': 'FUR-BO-10000112', 'Sauder Camden County Barrister Bookcase, Planked Cherry Finish': 'FUR-BO-10000330', 'Sauder Inglewood Library Bookcases': 'FUR-BO-10000362', "O'Sullivan 2-Shelf Heavy-Duty Bookcases": 'FUR-BO-10000468', 'Hon Metal Bookcases, Gray': 'FUR-BO-10000711', "O'Sullivan Plantations 2-Door Library in Landvery Oak": 'FUR-BO-10000780', "O'Sullivan Living Dimensions 2-Shelf Bookcases": 'FUR-BO-10001337', "O'Sullivan 3-Shelf Heavy-Duty Bookcases": 'FUR-BO-10001519', 'Bush Westfield Collection Bookcases, Dark Cherry Finish, Fully Assembled': 'FUR-BO-10001567', 'Sauder Mission Library with Doors, Fruitwood Finish': 'FUR-BO-10001601', 'Hon Metal Bookcases, Black': 'FUR-BO-10001608', "O'Sullivan Cherrywood Estates Traditional Bookcase": 'FUR-BO-10001619', 'Bush Somerset Collection Bookcase': 'FUR-BO-10001798', 'Atlantic Metals Mobile 5-Shelf Bookcases, Custom Colors': 'FUR-BO-10001811', 'Sauder Forest Hills Library with Doors, Woo

Creating Order Table

In [201]:
def step15_create_order_table(data_filename, db_file):

    def insert_orders(conn, values):
        sql = """
        INSERT INTO Orders (OrderID, CustomerID, ProductID, OrderDate, Quantity, Sales)
        VALUES (?, ?, ?, ?, ?, ?)
        """
        conn.executemany(sql, values)

    # Get mapping dictionaries
    customer_map = step8_create_customer_to_customerid_dictionary(db_file)
    product_map = step15_create_product_to_productid_dictionary(db_file)

    # Extract and map data
    orders = set()
    with open(data_filename, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            order_id = row['Order ID'].strip()
            customer_name = row['Customer Name'].strip()
            product_name = row['Product Name'].strip()
            order_date_raw  = row['Order Date'].strip()
            order_date = datetime.strptime(order_date_raw, "%d-%m-%Y").strftime("%Y-%m-%d")
            quantity = int(row['Quantity'])
            sales = float(row['Sales'])

            customer_id = customer_map.get(customer_name)
            product_id = product_map.get(product_name)

            if customer_id and product_id:
                orders.add((order_id, customer_id, product_id, order_date, quantity, sales))

    orders = sorted(list(orders), key=lambda x: x[0])  # Sort by OrderID

    # Create Orders table
    create_orders_table_sql = """
    CREATE TABLE Orders (
        OrderID TEXT NOT NULL PRIMARY KEY,
        CustomerID TEXT NOT NULL,
        ProductID TEXT NOT NULL,
        OrderDate TEXT NOT NULL,
        Quantity INTEGER NOT NULL,
        Sales REAL NOT NULL,
        FOREIGN KEY (CustomerID) REFERENCES Customer(CustomerID),
        FOREIGN KEY (ProductID) REFERENCES Product(ProductID)
    );
    """

    # Execute
    conn = create_connection(db_file)
    with conn:
        create_table(conn, create_orders_table_sql, "Orders")
        insert_orders(conn, orders)
    conn.close()

    print("Orders table created and populated.")


In [203]:
step15_create_order_table("Sales_data.csv", "normalized_sales.db")

Orders table created and populated.


In [205]:
conn = sqlite3.connect("normalized_sales.db")
df = pd.read_sql_query("select * from Orders", conn)
display(df)
conn.close()

,OrderID,CustomerID,ProductID,OrderDate,Quantity,Sales
0,1,937,FUR-BO-10001798,2017-11-08,4,3604.243977
1,10,641,OFF-AP-10002892,2015-06-09,9,7418.467726
2,100,3678,OFF-PA-10003256,2017-08-29,23,21636.623320
3,1000,3712,FUR-FU-10001918,2016-10-28,23,9630.103213
4,1001,1661,OFF-AR-10002956,2017-11-13,25,18766.519420
...,...,...,...,...,...,...
9660,994,3758,OFF-PA-10003625,2017-08-26,16,7467.099875
9661,995,3176,OFF-BI-10003925,2015-05-21,20,13152.230240
9662,997,3712,OFF-EN-10003862,2016-10-28,3,210.604021
9663,998,3712,OFF-ST-10004258,2016-10-28,3,2445.048934


Five questions about the dataset that the stakeholder would want
Q1: What is the total yearly sales trend over time?
Q2: Which product category generates the most revenue?
Q3: What are the top 5 customers based on total purchases?
Q4: What is the average order value per customer?
Q5: Which region contributed the most to sales in the most recent year?

Q1: What is the total yearly sales trend over time?

In [213]:
conn = create_connection("normalized_sales.db")
query1 = """
            SELECT 
                strftime('%Y', OrderDate) AS Year,
                ROUND(SUM(Sales), 2) AS TotalSales
            FROM Orders
            GROUP BY Year
            ORDER BY Year;
"""
df1 = pd.read_sql_query(query1, conn)
df1.head()
conn.close()

,Year,TotalSales
0,2015,12679233.88
1,2016,12994414.97
2,2017,16322337.80
3,2018,20888915.63


Q2: Which product category generates the most revenue?

In [215]:
conn = create_connection("normalized_sales.db")
query2 = """
SELECT 
    c.Category,
    ROUND(SUM(o.Sales), 2) AS TotalSales
FROM Orders o
JOIN Product p ON o.ProductID = p.ProductID
JOIN SubCategory s ON p.SubCategoryID = s.SubCategoryID
JOIN Category c ON s.CategoryID = c.CategoryID
GROUP BY c.Category
ORDER BY TotalSales DESC;
"""
df2 = pd.read_sql_query(query2, conn)
df2.head()
conn.close()

,Category,TotalSales
0,Office Supplies,37883315.36
1,Furniture,13224661.67
2,Technology,11776925.25


Q3: What are the top 5 customers based on total purchases?

In [220]:
conn = create_connection("normalized_sales.db")
query3 = """
SELECT 
    c.FirstName || ' ' || c.LastName AS CustomerName,
    ROUND(SUM(o.Sales), 2) AS TotalSpent
FROM Orders o
JOIN Customer c ON o.CustomerID = c.CustomerID
GROUP BY o.CustomerID
ORDER BY TotalSpent DESC
LIMIT 5;
"""
df3 = pd.read_sql_query(query3, conn)
df3.head()
conn.close()

,CustomerName,TotalSpent
0,John Lee,272209.28
1,Seth Vernon,270807.21
2,William Brown,246339.93
3,Matt Abelman,231106.26
4,Clay Ludtke,229990.59


Q4: What is the average order value per customer?


In [240]:
conn = create_connection("normalized_sales.db")
query4 = """
        SELECT 
        c.FirstName || ' ' || c.LastName AS CustomerName,
        COUNT(o.OrderID) AS TotalOrders,
        ROUND(SUM(o.Sales), 2) AS TotalSpent,
        ROUND(AVG(o.Sales), 2) AS AvgOrderValue
        FROM Orders o
        JOIN Customer c ON o.CustomerID = c.CustomerID
        GROUP BY o.CustomerID
        ORDER BY AvgOrderValue DESC;
"""
df4 = pd.read_sql_query(query4, conn)
df4.head()
conn.close()

,CustomerName,TotalOrders,TotalSpent,AvgOrderValue
0,Stefanie Holloman,2,44031.06,22015.53
1,Anthony O'Donnell,1,16509.59,16509.59
2,Mitch Willingham,2,31378.70,15689.35
3,Jim Radford,3,39729.39,13243.13
4,Aaron Bergman,6,73542.05,12257.01


Q5: Which region contributed the most to sales in the most recent year?

In [243]:
conn = create_connection("normalized_sales.db")
query5 = """
    WITH LatestYear AS (
        SELECT MAX(strftime('%Y', OrderDate)) AS MaxYear FROM Orders
    )
    SELECT 
        r.Region,
        ROUND(SUM(o.Sales), 2) AS TotalSales
    FROM Orders o
    JOIN Customer c ON o.CustomerID = c.CustomerID
    JOIN State s ON c.StateID = s.StateID
    JOIN Region r ON s.RegionID = r.RegionID
    WHERE strftime('%Y', o.OrderDate) = (SELECT MaxYear FROM LatestYear)
    GROUP BY r.Region
    ORDER BY TotalSales DESC
    LIMIT 1;
"""
df5 = pd.read_sql_query(query5, conn)
df5.head()
conn.close()

,Region,TotalSales
0,East,6260612.6


Implementing the same queries, on the same tables using pandas in Python.

In [247]:
# Connect to the database
conn = sqlite3.connect("normalized_sales.db")

# Load all required tables
orders = pd.read_sql_query("SELECT * FROM Orders", conn)
customers = pd.read_sql_query("SELECT * FROM Customer", conn)
products = pd.read_sql_query("SELECT * FROM Product", conn)
subcategories = pd.read_sql_query("SELECT * FROM SubCategory", conn)
categories = pd.read_sql_query("SELECT * FROM Category", conn)
states = pd.read_sql_query("SELECT * FROM State", conn)
regions = pd.read_sql_query("SELECT * FROM Region", conn)

conn.close()

# Convert OrderDate to datetime format
orders['OrderDate'] = pd.to_datetime(orders['OrderDate'])

In [255]:
orders['Year'] = orders['OrderDate'].dt.year
q1_df = orders.groupby('Year')['Sales'].sum().reset_index().sort_values('Year')
q1_df.columns = ['Year', 'TotalSales']
q1_df['TotalSales'] = q1_df['TotalSales'].map('{:,.2f}'.format)
q1_df.head()

,Year,TotalSales
0,2015,"12,679,233.88"
1,2016,"12,994,414.97"
2,2017,"16,322,337.80"
3,2018,"20,888,915.63"


In [257]:
q2_df = (
    orders.merge(products, on="ProductID")
          .merge(subcategories, on="SubCategoryID")
          .merge(categories, on="CategoryID")
          .groupby("Category")["Sales"]
          .sum()
          .reset_index()
          .sort_values("Sales", ascending=False)
)
q2_df.columns = ['Category', 'TotalSales']
q2_df['TotalSales'] = q2_df['TotalSales'].map('{:,.2f}'.format)
q2_df.head()

,Category,TotalSales
1,Office Supplies,"37,883,315.36"
0,Furniture,"13,224,661.67"
2,Technology,"11,776,925.25"


In [263]:
orders['CustomerID'] = orders['CustomerID'].astype(str)
customers['CustomerID'] = customers['CustomerID'].astype(str)
orders_customers = orders.merge(customers, on="CustomerID")
orders_customers['CustomerName'] = orders_customers['FirstName'] + ' ' + orders_customers['LastName']

q3_df = (
    orders_customers.groupby("CustomerName")["Sales"]
                    .sum()
                    .reset_index()
                    .sort_values("Sales", ascending=False)
                    .head(5)
)
q3_df.columns = ['CustomerName', 'TotalSales']
q3_df['TotalSales'] = q3_df['TotalSales'].map('{:,.2f}'.format)
q3_df.head()

,CustomerName,TotalSales
387,John Lee,"272,209.28"
689,Seth Vernon,"270,807.21"
787,William Brown,"246,339.93"
502,Matt Abelman,"231,106.26"
169,Clay Ludtke,"229,990.59"


In [265]:
q4_df = (
    orders_customers.groupby("CustomerName")
                    .agg(TotalOrders=('OrderID', 'count'),
                         TotalSpent=('Sales', 'sum'),
                         AvgOrderValue=('Sales', 'mean'))
                    .reset_index()
                    .sort_values("AvgOrderValue", ascending=False)
)
q4_df.head()

,CustomerName,TotalOrders,TotalSpent,AvgOrderValue
707,Stefanie Holloman,2,44031.060770,22015.530385
53,Anthony O'Donnell,1,16509.585510,16509.585510
547,Mitch Willingham,2,31378.701080,15689.350540
376,Jim Radford,3,39729.393664,13243.131221
0,Aaron Bergman,6,73542.053904,12257.008984


In [269]:
orders['Year'] = orders['OrderDate'].dt.year
latest_year = orders['Year'].max()

q5_df = (
    orders[orders['Year'] == latest_year]
    .merge(customers, on='CustomerID')
    .merge(states, on='StateID')
    .merge(regions, on='RegionID')
    .groupby('Region')['Sales']
    .sum()
    .reset_index()
    .sort_values('Sales', ascending=False)
    .head(1)
)
q5_df.columns = ['Region', 'TotalSales']
q5_df['TotalSales'] = q5_df['TotalSales'].map('{:,.2f}'.format)
q5_df.head()

,Region,TotalSales
1,East,"6,260,612.60"
